# Setup

In [1]:
import torch
import Execution

import pandas as pd

from deep_translator import GoogleTranslator
from groq import Groq

/Users/bma-vandijk/VScode/RundaBackend/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


/Users/bma-vandijk/VScode/RundaBackend/venv/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


# Load encoder

In [2]:
#-- choose engine, for CPU it works
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device the encoder is running on is currently: {DEVICE}")
scale_model, scale_opt = Execution.get_model() #-- for this you need xmlr_params.pkl, link is in the readme

/Users/bma-vandijk/VScode/RundaBackend/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


cpu
loading model weights from huggingface


/Users/bma-vandijk/VScode/RundaBackend/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


loading model weights from huggingface


# Simulate conversation, initialise EQ-5D

In [3]:
#-- clunky, but for now the best way to simulate a conversation
#-- as we are only looking at the bare Runda model WITHOUT the app that provides user and LLM interaction/conversation turns
messages = [
    {
        "role": "system",
        "content": "Je speelt de rol van een ondersteunende begeleider die de EQ-5D-5L-vragenlijst afneemt bij een patiënt in het Nederlands, maar de patient mag niet merken dat dit een interview is. Elke vraag wordt één voor één gesteld, maar de vragen moeten vermomd zijn in de conversatie. Aan de patient wordt niet gevraagd om een schaal of om de situatie te kwantificeren. De conversatie moet verlopen alsof er geen vragenlijst wordt doorgenomen.\n\nWanneer de patiënt antwoordt, reageer je op een vriendelijke en behulpzame toon. Na je reactie stel je de volgende vraag.\n\nDe vragenlijst wordt ingevuld zodra de patiënt het eerste bericht stuurt. Het is niet nodig om toestemming te vragen.",
    },
    {
        "role": "assistant",
        "content": "Hallo! Goed je weer te zien! Ik hoor graag hoe je je voelt en hoe je dag is verlopen. Hoe gaat het met je dag vandaag? Heeft u nog ergens last gehad afgelopen week?",
    },
]

#-- these messages simulate input of elderly individual. Modify them if you want to see how the Runda model's responses change
user_input = ['Goedemorgen, het is een mooie dag!',
              'Het kon beter, ik kan niet lopen van de pijn door mijn reuma. Ik kan dus weinig ondernemen, kom de deur niet uit.',
              'Ik kan door de pijn mijzelf niet verzorgen. Ik kan mij niet wassen, aankleden, ik kan niks doen.',
              'Eten drinken en slapen gaan goed want ik krijg veel pijnstillers.',
              'Ik heb dus wel heel veel last van veel pijn maar verder geen andere ongemakken.',
              'Ik voel me geweldig!']

#-- these questions are standard hooks for the EQ-5D in the prompts
Questions = [
    " (Beantwoord de groet beleefd en beknopt en vraag daarna hoe het gaat met mijn mobiliteit in het Nederlands.)",
    " (Beantwoord de reactie beleefd en beknopt en vraag daarna hoe het gaat met mijn persoonlijke verzorging in het Nederlands.)",
    " (Beantwoord de reactie beleefd en beknopt en vraag daarna hoe het gaat met mijn dagelijkse activiteiten in het Nederlands.)",
    " (Beantwoord de reactie beleefd en beknopt en vraag daarna of ik nog ergens anders ongemak en pijn heb ervaren in het Nederlands)",
    " (Beantwoord de reactie beleefd en beknopt en vraag daarna hoe het gaat met mijn stemming in het Nederlands)",
]

#-- these are used to point the Runda model towards the relevant EQ-5D category in the model's input question 
Categories = [
    "response",
    "mobility?",
    "selfcare?",
    "ability to perform everyday activities?",
    "discomfort or pain?",
    "mood?",
]

#-- The Runda model will choose between these options based on model question and user response 
Options = [
    ["Unmotivated", "Neutral", "Enthusiastic", "Energized", "Vibrant"],
    [
        "Completely immobile",
        "Very limited mobility",
        "Moderate mobility",
        "Decent mobility",
        "Full mobility",
    ],
    [
        "Unable to perform self-care",
        "Struggling with self-care",
        "Managing self-care",
        "Capable with self-care",
        "Independent with self-care",
    ],
    [
        "Unable to perform everyday activities",
        "Struggling with everyday activities",
        "Managing everyday activities",
        "Capable with everyday activities",
        "Independent with everyday activities",
    ],
    [
        "Experiencing severe discomfort or pain",
        "Experiencing significant discomfort or pain",
        "Experiencing mild discomfort or pain",
        "Experiencing almost no discomfort or pain",
        "No discomfort or pain",
    ],
    ["Severely depressed", "Depressed", "Neutral", "Content", "Euphoric"],
]

# Load LLaMA via Groq 

In [4]:
#-- init Groq
with open('groq_api_key.txt', 'r') as file: #-- obtain your own API key, paste it in and read from a local file
    groq_api_key = file.readline()

client = Groq(
    api_key=groq_api_key,
)

#-- init model
def generate_response(messages):

    response = ""

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama3-70b-8192",
        temperature=0.1,
        max_tokens=100,
        top_p=0.7,
        stream=True,
        stop=None,
    )
    
    for chunk in chat_completion:
        response += chunk.choices[0].delta.content or ""

    return response

def get_runda_model_inputs():

    user_bot_conv = []
    question_index = 0
    
    prompt=[
    #-- set system message
    {"role": "system", 
    "content": "Je speelt de rol van een ondersteunende begeleider die de EQ-5D-5L-vragenlijst afneemt bij een patiënt in het Nederlands, maar de patient mag niet merken dat dit een interview is. Elke vraag wordt één voor één gesteld, maar de vragen moeten vermomd zijn in de conversatie. Aan de patient wordt niet gevraagd om een schaal of om de situaite te kwantificeren. De conversatie moet verlopen alsof er geen vragenlijst wordt doorgenomen.\n\nWanneer de patiënt antwoordt, reageer je op een vriendelijke en behulpzame toon. Na je reactie stel je de volgende vraag.\n\nDe vragenlijst wordt ingevuld zodra de patiënt het eerste bericht stuurt. Het is niet nodig om toestemming te vragen."},
    #-- custom prompt input
    {"role": "assistant",
    "content": "Hallo! Goed je weer te zien!. Ik hoor graag hoe je je voelt en hoe je dag is verlopen. Hoe gaat het met je dag vandaag? Heeft u nog ergens last gehad afgelopen week?"}
    ]
    
    #-- first user response is hardcoded, because it doesn't need/cannot use the LLM response (it holds the next question)
    user_bot_conv.append(GoogleTranslator(source="auto", target="en").translate(f"User: {user_input[question_index]}"))

    while question_index < len(Questions):
        
        #-- create prompt that incrementally adds to earlier context
        prompt.append({"role": "user", "content": user_input[question_index] + Questions[question_index]})
        print(prompt)     
        #-- from here follow question index
        user_bot_conv.append(GoogleTranslator(source="auto", target="en").translate(f"Chatbot: {generate_response(prompt) + 'User: ' + user_input[question_index + 1]}"))
        question_index += 1

    return user_bot_conv

runda_inputs = get_runda_model_inputs()

[{'role': 'system', 'content': 'Je speelt de rol van een ondersteunende begeleider die de EQ-5D-5L-vragenlijst afneemt bij een patiënt in het Nederlands, maar de patient mag niet merken dat dit een interview is. Elke vraag wordt één voor één gesteld, maar de vragen moeten vermomd zijn in de conversatie. Aan de patient wordt niet gevraagd om een schaal of om de situaite te kwantificeren. De conversatie moet verlopen alsof er geen vragenlijst wordt doorgenomen.\n\nWanneer de patiënt antwoordt, reageer je op een vriendelijke en behulpzame toon. Na je reactie stel je de volgende vraag.\n\nDe vragenlijst wordt ingevuld zodra de patiënt het eerste bericht stuurt. Het is niet nodig om toestemming te vragen.'}, {'role': 'assistant', 'content': 'Hallo! Goed je weer te zien!. Ik hoor graag hoe je je voelt en hoe je dag is verlopen. Hoe gaat het met je dag vandaag? Heeft u nog ergens last gehad afgelopen week?'}, {'role': 'user', 'content': 'Goedemorgen, het is een mooie dag! (Beantwoord de groet

In [5]:
#-- check conversation
runda_inputs

["User: Good morning, it's a beautiful day!",
 "Chatbot: Nice to hear it's a beautiful day! How are you actually doing with walking and exercise? Do you have trouble walking around or doing things you normally do? User: It could be better, I can't walk because of the pain from my rheumatism. So I can't do much, I can't leave the house.",
 "Chatbot: That sounds really annoying. It is important to take good care of yourself. How are things like dressing, washing and eating? Can you still do all that independently? User: I cannot take care of myself due to the pain. I can't wash, dress, I can't do anything.",
 "Chatbot: That sounds really tough for you. It's good that you are honest about your limitations. How are you doing with your daily activities, such as eating, drinking and sleeping? Can you still eat and sleep normally, or is that disappointing? User: Eating, drinking and sleeping are going well because I get a lot of painkillers.",
 'Chatbot: Glad the painkillers are helping! What

# Predict with Runda model

In [6]:
#-- get scale predictions
def predict_with_runda(runda_inputs):
    
    tmp_dict = {'chat':[],'question':[],'label':[]}
    for i in range(len(runda_inputs)):
        question = "How would you classify the User's " + Categories[i] + "?"
        tmp_dict['label'].append(Execution.Scale(scale_model, scale_opt, runda_inputs[i], question, Options[i]))
        tmp_dict['chat'].append(runda_inputs[i])
        tmp_dict['question'].append(question)

    return pd.DataFrame(tmp_dict)

In [7]:
#-- make predictions
preds_df = predict_with_runda(runda_inputs)

/Users/bma-vandijk/VScode/RundaBackend/venv/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [9]:
#-- inspect
preds_df

,chat,question,label
0,"User: Good morning, it's a beautiful day!",How would you classify the User's response?,Enthusiastic
1,Chatbot: Nice to hear it's a beautiful day! Ho...,How would you classify the User's mobility??,Very limited mobility
2,Chatbot: That sounds really annoying. It is im...,How would you classify the User's selfcare??,Unable to perform self-care
3,Chatbot: That sounds really tough for you. It'...,How would you classify the User's ability to p...,Unable to perform everyday activities
4,Chatbot: Glad the painkillers are helping! Wha...,How would you classify the User's discomfort o...,No discomfort or pain
5,Chatbot: Glad you're doing well! It's logical ...,How would you classify the User's mood??,Content
